In [54]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [55]:
url = 0
reg_data = pd.read_csv(r'C:\Users\abhyu\Desktop\Dissertation\Code\fetchDetailedUserData\detailedREGUserData_FILTERED_NEW.csv')
reg_list = reg_data['movies watched'].to_list()
reg_url_list = []

pop_data = pd.read_csv(r'C:\Users\abhyu\Desktop\Dissertation\Code\fetchDetailedUserData\detailedPOPUserData_FILTERED_NEW.csv')
pop_list = pop_data['movies watched'].to_list()
pop_url_list = []

for user in reg_list:
    for index, film in enumerate(eval(user)):
        reg_url_list.append(film['url'])


print('pop')
for user in pop_list:
        for index, film in enumerate(eval(user)):
            pop_url_list.append(film['url'])
    

url_list = pop_url_list + reg_url_list

url_list = list(set(url_list))
len(url_list)  #51789
# url_list

pop


10185

In [56]:

done_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\Code\fetchUserRelevantMovieData\df_till8303.csv")
done_data_list = done_data['11'].to_list()
done_data_list
print(len(url_list))
new_list = [item for item in url_list if item not in done_data_list]
len(new_list)

10185


1882

In [57]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}


In [58]:
def parse_page(url = url, headers= headers):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    # print(soup)
    return soup

In [59]:
#data for film in individual page

def film_data(url, film_soup):
    try:
        title = film_soup.find('h1', class_="headline-1 js-widont prettify").text
    except:
        title = None

    try:
        year_of_release = int(film_soup.find('small', class_="number").text)
    except:
        year_of_release = None
        
    try:
        actor_data = film_soup.find_all('a', {"class" : "text-slug tooltip"}, limit= 3)
        actors = []
        for actor in actor_data:
            actors.append(actor.text)
    except:
        actor_data = None

    try:
        director = film_soup.find('span', {"class" : "prettify"}).text
    except:
        director = None
    # country_or_countries = film_soup.find()
    if film_soup.find('h3', text=re.compile(r'Country|Countries')):
        country_tag = film_soup.find('h3', text=re.compile(r'Country|Countries')).find_next('div')
        country = [a.text for a in country_tag.find_all('a')]
    else:
        country = None

    try:
        language = film_soup.find('h3', text=re.compile(r'Original Language|Language')).find_next('div').text.strip()
    except:
        language = None

    try:
        rating = film_soup.find('meta', {"name" : "twitter:data2"})["content"]
        rating = float(rating.replace(' out of 5','')) * 2
    except:
        rating = None

    try:
        stat_soup = parse_page(url + '/members')

        letterboxd_views = stat_soup.find('a', class_="tooltip")['title']
        letterboxd_views = int(''.join(filter(str.isdigit, letterboxd_views)))
    except:
        letterboxd_views = 0
    # letterboxd_views = None

    #Genre
    try:
        genres_data = film_soup.find('div', class_="text-sluglist capitalize")
        genres = []
        for genre in genres_data.find_all('a'):
            genres.append(genre.text.strip())
    except:
        genres = None

    #Number of films in the franchise
    if film_soup.find('section', {"id" : "related"}):
        franchise = "yes"
    else:
        franchise = "no"

    try:
        film_length = film_soup.find('p',class_="text-link text-footer").text
        film_length = int(''.join(filter(str.isdigit, film_length)))
    except:
        film_length= None
    #LINKS FOR OTHER SITES
    try:
        imdb_link = film_soup.find('a', {"data-track-action" : "IMDb"})['href']
    except:
        imdb_link = None
    try:
        tmdb_link = film_soup.find('a', {"data-track-action" : "TMDb"})['href']
    except:
        tmdb_link = None
        
    film_data_list = [title, year_of_release, film_length, rating, letterboxd_views, director, country, language, actors, genres, franchise, url, imdb_link, tmdb_link]
    # more_film_list.append(actors)

    
    return film_data_list

In [60]:
def list_iteration(url_list = url_list, headers = headers):
    final_df = pd.DataFrame()
    for index, url in enumerate(url_list):

        soup = parse_page(url, headers)
        print(url)
        data_list = film_data(url, soup)
        df_row = pd.DataFrame(data_list)
        final_df = pd.concat([final_df, df_row], axis=1)
        print(index)

        if index%100 == 0:
            final_df.transpose().to_csv(fr'C:\Users\abhyu\Desktop\Dissertation\Code\fetchUserRelevantMovieData\df_final.csv', index = False)

        #Testing
        # if index == 2:
        #     break
        
    return final_df.transpose()

In [61]:
print(len(new_list))
df = list_iteration(new_list, headers)
df

1882
https://letterboxd.com/film/the-treasure-of-the-sierra-madre/
0
https://letterboxd.com/film/the-taking-of-deborah-logan/
1
https://letterboxd.com/film/soft-shoe/
2
https://letterboxd.com/film/heartburn/
3
https://letterboxd.com/film/brian-and-charles-2022/
4
https://letterboxd.com/film/sound-of-metal/
5
https://letterboxd.com/film/the-secret-garden-1988/
6
https://letterboxd.com/film/10-things-i-hate-about-you/
7
https://letterboxd.com/film/three-ages/
8
https://letterboxd.com/film/night-at-the-museum-secret-of-the-tomb/
9
https://letterboxd.com/film/happy-halloween-scooby-doo/
10
https://letterboxd.com/film/creating-the-queens-gambit/
11
https://letterboxd.com/film/clerks-iii/
12
https://letterboxd.com/film/cinderella-liberty/
13
https://letterboxd.com/film/jason-x/
14
https://letterboxd.com/film/tribute-to-zgougou-the-cat/
15
https://letterboxd.com/film/sherlock-the-great-game-2010/
16
https://letterboxd.com/film/first-cow/
17
https://letterboxd.com/film/mutt-2023/
18
https://le

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,The Treasure of the Sierra Madre,1948,126,8.36,76577,John Huston,[USA],English,"[Humphrey Bogart, Walter Huston, Tim Holt]","[adventure, drama, western]",no,https://letterboxd.com/film/the-treasure-of-th...,http://www.imdb.com/title/tt0040897/maindetails,https://www.themoviedb.org/movie/3090/
0,The Taking of Deborah Logan,2014,90,6.18,67199,Adam Robitel,[USA],English,"[Jill Larson, Anne Ramsay, Michelle Ang]","[horror, mystery, thriller]",no,https://letterboxd.com/film/the-taking-of-debo...,http://www.imdb.com/title/tt3387648/maindetails,https://www.themoviedb.org/movie/297608/
0,Soft Shoe,1987,20,None,43,Holly Fisher,None,French,[],None,no,https://letterboxd.com/film/soft-shoe/,None,https://www.themoviedb.org/movie/552720/
0,Heartburn,1986,108,6.06,12580,Mike Nichols,[USA],English,"[Meryl Streep, Jack Nicholson, Jeff Daniels]","[romance, drama, comedy]",no,https://letterboxd.com/film/heartburn/,http://www.imdb.com/title/tt0091188/maindetails,https://www.themoviedb.org/movie/13818/
0,Brian and Charles,2022,90,7.08,17110,Jim Archer,[UK],English,"[David Earl, Chris Hayward, Louise Brealey]","[comedy, science-fiction, drama]",no,https://letterboxd.com/film/brian-and-charles-...,http://www.imdb.com/title/tt13270424/maindetails,https://www.themoviedb.org/movie/913814/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Pool of London,1951,85,7.12,1072,Basil Dearden,[UK],English,"[Bonar Colleano, Susan Shaw, Renée Asherson]","[crime, drama]",no,https://letterboxd.com/film/pool-of-london/,http://www.imdb.com/title/tt0042851/maindetails,https://www.themoviedb.org/movie/27917/
0,Guess Who's Coming to Dinner,1967,108,7.5,43983,Stanley Kramer,[USA],English,"[Katharine Houghton, Sidney Poitier, Katharine...","[comedy, drama]",no,https://letterboxd.com/film/guess-whos-coming-...,http://www.imdb.com/title/tt0061735/maindetails,https://www.themoviedb.org/movie/1879/
0,Fear City: A Family-Style Comedy,1994,93,7.74,14821,Alain Berbérian,[France],French,"[Chantal Lauby, Alain Chabat, Dominique Farrugia]",[comedy],no,https://letterboxd.com/film/fear-city-a-family...,http://www.imdb.com/title/tt0109440/maindetails,https://www.themoviedb.org/movie/15097/
0,Fatal Deviation,1998,77,5.62,1160,Shay Casserley,[Ireland],English,"[James P. Bennett, Mikey Graham, Conor O'Shea]",[action],no,https://letterboxd.com/film/fatal-deviation/,http://www.imdb.com/title/tt0488046/maindetails,https://www.themoviedb.org/movie/88909/


In [62]:
df.to_csv(fr'C:\Users\abhyu\Desktop\Dissertation\Code\fetchUserRelevantMovieData\df_final.csv', index = False)

In [69]:
done_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\Code\fetchUserRelevantMovieData\df_till8303.csv")

new_done_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\Code\fetchUserRelevantMovieData\df_final.csv")
final_data = pd.concat([done_data, new_done_data])
final_data
final_data.to_csv(fr'C:\Users\abhyu\Desktop\Dissertation\Code\fetchUserRelevantMovieData\finalMovieData.csv', index = False)
# new_done_data